In [9]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

pd.options.display.max_rows = 999

In [10]:
df = pd.read_csv('parking_citations_unc.csv')
# df_corrupt = pd.read_csv('parking_citations_cor.csv')

In [16]:
top_25 = df['Make'].value_counts().index[:25]
df['top_25'] = np.where(df['Make'].isin(top_25),1,0)
df[df['top_25']==1]

,Unnamed: 0,Ticket number,Issue time,Marked Time,Plate Expiry Date,VIN,Agency,Fine amount,Latitude,Longitude,...,Violation Description_STANDNG IN ALLEY,Violation Description_STNDNG IN ALLEY,Violation Description_STOP/STAND PROHIBIT,Violation Description_TIME LIMIT/CITY LOT,Violation Description_WHITE CURB,Violation Description_WHITE ZONE,Violation Description_WITHIN INTERSECTION,Violation Description_YELLOW ZONE,Make,top_25
0,4,1105461453,115.0,0.0,200316.0,0.0,1.0,93.0,99999.0,99999.0,...,0,0,0,0,0,0,0,0,CHEV,1
1,6,1106500452,1710.0,0.0,201605.0,0.0,1.0,163.0,99999.0,99999.0,...,0,0,0,0,0,0,0,0,MAZD,1
2,10,1106506424,1100.0,0.0,201511.0,0.0,1.0,93.0,99999.0,99999.0,...,0,0,0,0,0,0,0,0,FORD,1
3,11,1106506435,1105.0,0.0,201701.0,0.0,1.0,93.0,99999.0,99999.0,...,0,0,0,0,0,0,0,0,CHRY,1
4,14,1107179581,1055.0,0.0,201605.0,0.0,54.0,68.0,99999.0,99999.0,...,0,0,0,0,0,0,0,0,TOYO,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,4993,4273834806,819.0,0.0,201604.0,0.0,53.0,63.0,6426293.5,1892073.3,...,0,0,0,0,0,0,0,0,BMW,1
2481,4995,4273834821,1046.0,0.0,0.0,0.0,53.0,63.0,6425965.9,1890974.4,...,0,0,0,0,0,0,0,0,CHRY,1
2482,4996,4273834832,1053.0,0.0,201606.0,0.0,53.0,63.0,6425907.7,1889884.4,...,0,0,0,0,0,0,0,0,TOYT,1
2483,4997,4273834843,1056.0,0.0,201603.0,0.0,53.0,63.0,6426349.4,1889152.6,...,0,0,0,0,0,0,0,0,NISS,1


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Make','top_25']), df['top_25'], test_size=0.3)

In [5]:
y_train

6       1
153     0
1936    1
2068    1
1486    1
       ..
809     1
1800    1
835     1
2035    1
1360    1
Name: top_25, Length: 1739, dtype: int64

In [17]:
lr = LogisticRegression(class_weight='auto')
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight='auto', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
lr.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [19]:
y_test

2477    1
1540    1
1159    0
1138    1
1592    1
2350    1
2447    1
245     1
832     1
1917    1
1531    1
551     0
134     0
368     1
665     1
405     1
1851    1
48      1
2062    1
1163    1
2397    1
1383    1
1078    0
1087    1
2172    1
1017    1
1295    1
190     0
912     1
2416    1
1749    1
1923    1
1221    1
917     1
441     0
631     1
1405    0
1931    1
216     1
1579    1
733     0
1339    1
1083    1
1000    1
2086    1
2030    0
604     1
680     1
477     1
2226    1
1714    1
1591    1
948     1
1387    1
944     1
27      1
1528    1
2296    1
254     0
1388    1
2421    1
1130    1
1379    1
1317    1
1175    1
2215    1
1406    0
537     1
522     1
1490    1
910     1
1850    1
723     1
1443    1
539     1
257     1
1481    1
1407    1
669     1
398     1
2202    0
2134    1
2471    0
300     1
1234    1
194     1
2182    1
2454    1
1502    1
1729    1
1249    1
2001    1
865     1
1385    1
589     1
1133    1
53      1
2378    0
2       1
788     1
